In [31]:
from __future__ import print_function
import torch
import torch.nn as nn
import math
import numpy as np
from functools import reduce

In [2]:
"""
n = dimension of input
m = dimension of output
"""

'\nn = dimension of input\nm = dimension of output\n'

In [3]:
def calc_logit(input, weight, bias=None):
    """
    Calculate logit = input.mm(weight.t()) + bias

    Args:
        input:  [1 x n]         Input vector
        weight:  [m x n]        Weight matrix
        bias:  [m]              Bias vector

    Returns:
        logit: [1 x n]          Logit = input.mm(weight.t()) + bias

    """

    logit = input.mm(weight.t())
    if bias is not None:
        logit += bias.unsqueeze(0).expand_as(logit)

    return logit

## ESGD

In [7]:
def esgd_arctan(grad_output, input, weight, bias, output):

    grad_output_scaled = grad_output / (1 + logit ** 2)  # [1 x m]
    grad_input = grad_output_scaled.mm(weight)  # [1 x n]
    grad_weight = grad_output_scaled.t().mm(input)  # [m x n]
    grad_bias = grad_output_scaled.sum(0).squeeze(0)  # [m]

    return grad_input, grad_weight, grad_bias

In [8]:
esgd_arctan(grad_output, input, weight, bias, output)

(
  0.3953  0.2813
 [torch.FloatTensor of size 1x2], 
  0.1423 -0.0742
 -0.6416  0.3346
  0.0886 -0.0462
 [torch.FloatTensor of size 3x2], 
  0.1608
 -0.7249
  0.1001
 [torch.FloatTensor of size 3])

## ISGD

In [13]:
def calc_backwards_variables(saved_tensors, logit, grad_output, lr, mu):
    """
    Calculate the variables required for back-propagation
    
    Args:
        saved_tensors:          Stores from forward-propagation the input, weight, bias, output
        logit: [1 x n]          Stores from forward-propagation the logit
        grad_output: [1 x m]    The gradient that has been back-propagated to this layer
        lr: [1]                 Learning rate
        mu: [1]                 Ridge-regularization constant

    Returns:
        input: [1 x n]          Input vector
        weight: [m x n]         Weight matrix
        bias: [m]               Bias vector
        output [1 x m]          Input to the next layer = logit put through the non-linear activation function
        logit: [1 x n]          Logit
        s: [1 x m]              Sign of back-propagated gradient
        z_norm: [1]             2-norm of (input, 1)
        d: [1 x m]              Weighted constant, proportional to the sqrt(abs(back-propagated gradient))
        c: [1 x m]              Logit contracted by ridge-regularization
    """

    # Unpack saved values
    input, weight, bias, output = saved_tensors

    # ISGD constants
    s = torch.sign(grad_output)
    z_norm = math.sqrt((torch.norm(input) ** 2 + 1.0))
    d = z_norm / math.sqrt(1.0 + lr * mu) * torch.sqrt(torch.abs(grad_output))
    c = logit / (1.0 + lr * mu)

    return input, weight, bias, output, logit, s, z_norm, d, c

In [12]:
def calc_weigh_bias_grad(weight, mu, lr, a, d, input, z_norm, bias):
    """
    Calculate the gradient of the weight matrix and bias vector

    Args:
        weight: [m x n]         Weight matrix
        mu: [1]                 Ridge-regularization constant
        lr: [1]                 Learning rate
        a: [1 x m]              Solution of ISGD update
        d: [1 x m]              Weighted constant, proportional to the sqrt(abs(back-propagated gradient))
        input: [1 x n]          Input vector
        z_norm: [1]             2-norm of (input, 1)
        bias: [m]               Bias vector

    Returns:
        grad_weight: [m x n]    Gradient of the weight matrix
        grad_bias: [m]          Gradient of the bias vector

    """

    grad_weight = weight * mu / (1.0 + lr * mu) - (a * d).t().mm(input) / z_norm ** 2
    grad_bias = bias * mu / (1.0 + lr * mu) - (a * d).squeeze() / z_norm ** 2
    return grad_weight, grad_bias

In [149]:
def real_root_closest_to_zero(coeff):
    roots = np.roots(coeff)
    real_roots = [root.real for root in roots if root.imag == 0]
    root_closest_to_zero = reduce((lambda x, y: x if (abs(x) < abs(y)) else y), real_roots)
    return root_closest_to_zero

In [77]:
def isgd_arctan(grad_output, input, weight, bias, output, logit):
    
    lr, mu = 0.00000001, 0.0
    input, weight, bias, output, logit, s, z_norm, d, c = calc_backwards_variables([input, weight, bias, output], logit,
                                                                                       grad_output, lr, mu)

    # Calculate a
        coeff = np.array([((lr * d) ** 2).numpy()[0],
                          (2 * lr * d * c).numpy()[0],
                          (c ** 2 + 1).numpy()[0],
                          (s * d).numpy()[0]])

        root_closest_to_zero = np.apply_along_axis(real_root_closest_to_zero, 0, coeff)
        a = torch.from_numpy(root_closest_to_zero).unsqueeze(1).t().type(torch.FloatTensor)

        # Calculate grad_weight, grad_bias and return all gradients
        grad_weight, grad_bias = calc_weigh_bias_grad(weight, mu, lr, a, d, input, z_norm, bias)

        # Calculate input gradient
        grad_output_scaled = grad_output / (1 + logit ** 2)  # [1 x m]
        grad_input = grad_output_scaled.mm(weight)  # [1 x n]

        return grad_input, grad_weight, grad_bias

In [152]:
def isgd_arctan2(grad_output, input, weight, bias, output, logit):
    
    lr, mu = 0.00000001, 0.0
    input, weight, bias, output, logit, s, z_norm, d, c = calc_backwards_variables([input, weight, bias, output], logit,
                                                                                       grad_output, lr, mu)

    # Calculate a
    a = d*0
    diff = 1
    iter_count = 0
    while diff < 1e-10:
        a_new = - s * d / (1.0 + (lr * d * a + c)**2)
        diff = torch.norm(a - a_new)
        a = a_new
        iter_count += 1
        assert(iter_count < 20), 'Arctan update has failed to converge'

    # Calculate grad_weight, grad_bias and return all gradients
    grad_weight, grad_bias = calc_weigh_bias_grad(weight, mu, lr, a, d, input, z_norm, bias)

    # Calculate input gradient
    grad_output_scaled = grad_output / (1 + logit ** 2)  # [1 x m]
    grad_input = grad_output_scaled.mm(weight)  # [1 x n]

    return grad_input, grad_weight, grad_bias


In [185]:
# Calculate a
a = d*0
diff = 1
iter_count = 0
while diff < 1e10:
    a_new = - s * d / (1.0 + (lr * d * a + c)**2)
    diff = torch.norm(a - a_new)
    print(diff)
    a = a_new
    iter_count += 1
    assert(iter_count < 20), 'Arctan update has failed to converge'
    

1.1638623909718155
0.010818114720922343
0.00010266294157844973
9.5367431640625e-07
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0


AssertionError: bot

## Test the differences between ESGD and ISGD

In [170]:
# Random data
grad_output = torch.randn(1, 3)     # [1 x m]
input = torch.randn(1, 2)           # [1 x n]
weight = torch.randn(3, 2)          # [m x n]
bias = torch.randn(3,)              # [m]

# Forward propagation
# Calculate logit [1 x m], where logit = input.mm(weight.t()) + bias
logit = calc_logit(input, weight, bias)

# Non-linear activation function
output = torch.atan(logit)  # [1 x m]

# print('logit: ', logit)
# print('output: ', output)

# isgd_grads = isgd_arctan(grad_output, input, weight, bias, output, logit)
isgd_grads2 = isgd_arctan2(grad_output, input, weight, bias, output, logit)
esgd_grads = esgd_arctan(grad_output, input, weight, bias, output)

# print(isgd_grads)
# print(esgd_grads)
print([(x-y) for x,y in zip(isgd_grads2, esgd_grads)])

[
 0  0
[torch.FloatTensor of size 1x2]
, 
1.00000e-07 *
  0.0093  0.0000
 -0.2980 -0.5960
  0.8941  1.7881
[torch.FloatTensor of size 3x2]
, 
1.00000e-07 *
  0.0000
  0.5960
 -2.3842
[torch.FloatTensor of size 3]
]


# Full Pytorch module

## Forward propagation

In [ ]:
input = torch.randn(1, 3)
weight = torch.randn(5, 3)
bias = torch.randn(5,)
output = input.mm(weight.t())
output += bias.unsqueeze(0).expand_as(output)
relu = torch.clamp(output, min=0.0)

print('input: ', input)
print('weight: ', weight.size())
print('bias: ', bias.size())
print('output: ', output.size())
print('relu: ', relu.size())

## Back-propagation

## ISGD

In [ ]:
# Constants
s = torch.sign(grad_output)
abs_grad_output = torch.abs(grad_output)
# Note that torch.norm outputs a float instead of a tensor
z_norm = math.sqrt((torch.norm(input) ** 2 + 1.0))
d = z_norm * math.sqrt(lr/(1.0+lr*mu)) * torch.sqrt(abs_grad_output)
c = output/(1.0+lr*mu)
# print('s: ', s)
# print('delta: ', d) 
# print(c)

# Calculate alpha
alpha = alpha_relu(s,d,c)

# Calculate gradients
new_weight = weight / (1.0 + lr * mu) + alpha.mul(d).mm(weight) / z_norm**2
grad_weight = (weight - new_weight) / lr
# print(weight)
# print(new_weight)
# print(grad_weight)

new_bias = bias / (1.0 + lr * mu) + alpha.mul(d).squeeze().mul(bias) / z_norm**2
grad_bias = (bias - new_bias) / lr
# print(bias)
# print(new_bias)
# print(grad_bias)

sgn_output = (output >= 0).type(torch.FloatTensor)
grad_input = (grad_output.mul(sgn_output)).mm(weight)
print(grad_input)

In [ ]:
def alpha_relu(s,d,c):
#     cond1 = (s == 1).mul(c <= 0).type(torch.FloatTensor)
    cond2 = (s == 1).mul(c > 0).mul(c <= d**2).type(torch.FloatTensor)
    cond3 = (s == 1).mul(c > d**2).type(torch.FloatTensor)
#     cond4 = (s == -1).mul(c <= -d**2/2.0).type(torch.FloatTensor)
    cond5 = (s == -1).mul(c > -d**2/2.0).type(torch.FloatTensor)
    # print(cond1, cond2, cond3, cond4, cond5)

    alpha = (0.0
#              + 0.0 * cond1
            - (c.div(d)).mul(cond2)
            - d.mul(cond3)
#             + 0.0 * cond4
            + d.mul(cond5)
            )

    return alpha

In [ ]:
alpha_relu(s,d,c)

In [ ]:
# Understand grad_output_pos_out.sum(0).squeeze(0)
print(grad_output_pos_out)
print(grad_output_pos_out.sum(0))
print(grad_output_pos_out.sum(0).squeeze(0))

## Standard RELU

In [ ]:
pos_out = (output >= 0).type(torch.FloatTensor)
grad_output_pos_out = torch.mul(grad_output, pos_out)
grad_input = grad_output_pos_out.mm(weight)
grad_weight = grad_output_pos_out.t().mm(input)
grad_bias = grad_output_pos_out.sum(0).squeeze(0)

print('grad_output: ', grad_output.size())
print('output: ', output.size())
print('pos_out: ', pos_out.size())
print('grad_output_pos_out: ', grad_output_pos_out.size())
print('grad_input: ', grad_input.size())
print('grad_bias: ', grad_bias.size())
print('grad_weight: ', grad_weight.size())